In [1]:
from bs4 import BeautifulSoup as bs
import requests
from splinter import Browser
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd


def init_browser():
    executable_path = {"executable_path": ChromeDriverManager().install()}
    return Browser("chrome", **executable_path, headless=False)

def scrape():
    browser = init_browser()

################
### NASA Mars News
################
    url = "https://mars.nasa.gov/news/"
    response = requests.get(url)

# Create a BS object
    marsSite = bs(response.text, 'html.parser')

# Create News Title and Paragraph variables 
    news_title = marsSite.title.text
    news_p = marsSite.body.p.text
################
### JPL Mars Space Images - Featured Image
################
# Setup splinter
    executable_path = {'executable_path': ChromeDriverManager().install()}
    browser = Browser('chrome', **executable_path, headless=False)

    url = "https://data-class-jpl-space.s3.amazonaws.com/JPL_Space/index.html"
    browser.visit(url)

# HTML object
    html = browser.html
    featImg_site = bs(html, 'html.parser')

# Get Featured Image 'href'
    results = featImg_site.find('div', class_ = 'header')
    featA = results.find('a', class_ = 'showimg fancybox-thumbs')
    featImg_href = featA['href']

# Save complete url string for Featured Image
    partUrl = "https://data-class-jpl-space.s3.amazonaws.com/JPL_Space/"
    featImg_url = partUrl + featImg_href

    browser.quit()
################
### Mars Facts
################
# Scrape Mars Fact page for facts table
    url = 'https://space-facts.com/mars/'
    tables = pd.read_html(url)

# Convert scraped data to Pandas dataframe
    mars_df = tables[0]

# Convert dataframe to HTML table
    mars_htmlTable = mars_df.to_html()

################
### Mars Hemispheres
################
    url = "https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars"
    response = requests.get(url)
    marsHemi = bs(response.text, 'lxml')

# List of results
    results = marsHemi.find_all('a', class_ = "itemLink product-item")

# Loop through returned results
    executable_path = {'executable_path': ChromeDriverManager().install()}
    browser = Browser('chrome', **executable_path, headless=False)

    url = "https://astrogeology.usgs.gov"
    hemisphere_image_urls = []

    for result in results:
        dict = {}
        href = result['href']
        imgSite = url + href
        browser.visit(imgSite)
        html = browser.html
        featImg_site = bs(html, 'lxml')
        # Identify full-resolution image container
        title = featImg_site.title.text.strip()
        title = title.replace(" Enhanced | USGS Astrogeology Science Center", "")
        images = featImg_site.find('li')
        img_url = images.find('a')['href']
        dict['title'] = title
        dict['img_url'] = img_url
        hemisphere_image_urls.append(dict)

    scrape_data = {
        "news_title" : news_title,
        "news_p" : news_p,
        "featImg_url" : featImg_url,
        "mars_table" : mars_htmlTable,
        "hemisphere_image_urls" : hemisphere_image_urls
    }
    
    browser.quit()

    return scrape_data


In [2]:
scrape()



====== WebDriver manager ======
Current google-chrome version is 90.0.4430
Get LATEST driver version for 90.0.4430
Driver [C:\Users\nikoe\.wdm\drivers\chromedriver\win32\90.0.4430.24\chromedriver.exe] found in cache


====== WebDriver manager ======
Current google-chrome version is 90.0.4430
Get LATEST driver version for 90.0.4430
Driver [C:\Users\nikoe\.wdm\drivers\chromedriver\win32\90.0.4430.24\chromedriver.exe] found in cache


====== WebDriver manager ======
Current google-chrome version is 90.0.4430
Get LATEST driver version for 90.0.4430
Driver [C:\Users\nikoe\.wdm\drivers\chromedriver\win32\90.0.4430.24\chromedriver.exe] found in cache


{'news_title': 'News  – NASA’s Mars Exploration Program ',
 'news_p': 'Managed by the Mars Exploration Program and the Jet Propulsion Laboratory for NASA’s Science Mission Directorate',
 'featImg_url': 'https://data-class-jpl-space.s3.amazonaws.com/JPL_Space/image/featured/mars3.jpg',
 'mars_table': '<table border="1" class="dataframe">\n  <thead>\n    <tr style="text-align: right;">\n      <th></th>\n      <th>0</th>\n      <th>1</th>\n    </tr>\n  </thead>\n  <tbody>\n    <tr>\n      <th>0</th>\n      <td>Equatorial Diameter:</td>\n      <td>6,792 km</td>\n    </tr>\n    <tr>\n      <th>1</th>\n      <td>Polar Diameter:</td>\n      <td>6,752 km</td>\n    </tr>\n    <tr>\n      <th>2</th>\n      <td>Mass:</td>\n      <td>6.39 × 10^23 kg (0.11 Earths)</td>\n    </tr>\n    <tr>\n      <th>3</th>\n      <td>Moons:</td>\n      <td>2 (Phobos &amp; Deimos)</td>\n    </tr>\n    <tr>\n      <th>4</th>\n      <td>Orbit Distance:</td>\n      <td>227,943,824 km (1.38 AU)</td>\n    </tr>\n    <tr

NameError: name 'scrape_data' is not defined